In [1]:
train <- read.delim(file='train.tsv', header=TRUE, stringsAsFactors=FALSE)
test <- read.delim(file='test.tsv', header=TRUE, stringsAsFactors=FALSE)

In [4]:
test$Sentiment <- ""

In [5]:
combi <- rbind(train,test)


In [37]:
#Random Sampling of rows
combi_sample <- combi[sample(nrow(combi),10000),]

In [6]:
dim(combi)

[1] 222352      4

In [8]:
dim(combi_sample)

[1] 30000     4

In [9]:
DTM <- function(corpus) {
    dtm = DocumentTermMatrix(corpus,control=list(tolower=T,removePunctuation=T,removeNumbers=T,stopwords=T,
                                                stemming=T,weighting= function(x) weightTfIdf(x,normalize=T)))
}

In [40]:
corpus <- Corpus(VectorSource(combi_sample$Phrase))

In [32]:
#install.packages("SnowballC",repos = "http://cran.us.r-project.org",dependencies = TRUE)

Installing package into ‘/home/ubuntu/R/x86_64-pc-linux-gnu-library/3.2’
(as ‘lib’ is unspecified)



The downloaded source packages are in
	‘/tmp/RtmpAhvq96/downloaded_packages’


In [33]:
#library(SnowballC)

In [38]:
combi_sample$Phrase <- iconv(combi_sample$Phrase,to="utf-8")

In [39]:
combi_sample$Phrase <- combi_sample$Phrase[!is.na(combi_sample$Phrase)]

In [11]:
library(tm)

Loading required package: NLP


In [13]:
inspect(corpus[1])

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 1

[[1]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 29



In [16]:
inspect(corpus[1:3])

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 3

[[1]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 29

[[2]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 14

[[3]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 144



In [41]:
dtm_movie <- DTM(corpus)

Warning message:
In weightTfIdf(x, normalize = T): empty document(s): 73 145 347 409 411 514 599 740 807 1163 1195 1296 1330 1562 1649 1712 1718 1754 1790 1804 1838 1890 2039 2071 2331 2502 2665 2887 2939 2970 3137 3170 3195 3231 3243 3282 3482 3591 3601 3687 3816 3883 3938 4037 4073 4130 4155 4178 4243 4289 4325 4406 4550 4606 4608 4651 4770 4942 4991 5082 5133 5242 5285 5468 5578 5659 5664 5675 5772 5905 6119 6185 6444 6447 6455 6535 6715 6729 6835 6895 6916 7019 7346 7697 7793 7870 8101 8274 8322 8343 8437 8584 8614 8904 8921 9045 9059 9199 9303 9475 9640 9735 9858 9905 9993

In [42]:
dtm_phrase <- as.data.frame(as.matrix(dtm_movie))

In [43]:
colnames(dtm_phrase) <- paste("movie_",colnames(dtm_phrase),sep="")

In [53]:
length(combi_sample$SentenceID)

[1] 0

In [54]:
all_features <- data.frame(PhraseId = combi_sample$PhraseId,Sentiment = combi_sample$Sentiment,dtm_phrase)

In [55]:
head(all_features)

,PhraseId,Sentiment,movie_aaliyah,movie_abandon,movie_abba,movie_abc,movie_aberr,movie_abhorr,movie_abil,movie_abl,ellip.h,movie_zhang,movie_zhuangzhuang,movie_zigzag,movie_zip,movie_zipper,movie_zippi,movie_zish,movie_zombi,movie_zone,movie_zwick
1,190291,,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,108300,4,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,189164,,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,100272,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,153576,2,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,67578,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [57]:
sparse_train <- as.matrix(as.matrix(all_features[,3:ncol(all_features)]),sparse=T)

In [59]:
library(e1071)

In [60]:
#Support Vectors Machines to learn patterns in the data from all the IVs to predict DV
model <- svm(sparse_train,as.factor(all_features$Sentiment),kernel = "linear",cost = 0.3)

ERROR: Error: cannot allocate vector of size 576.4 Mb


In [ ]:
all_features$predict <- predict(model,sparse_train)